# Notebook Lecture 9: Frequency Response
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook introduces frequency response analysis methods for linear time-invariant systems, covering Bode plots, polar plots, and their applications in control system design.

Authors:
- Mark Benazet Castells; mbenazet@ethz.ch
- Felix Muller; fmuller@ethz.ch

## Learning Objectives

After completing this material, you should be able to:

### 1. Master Bode Plot Construction
- Recognize why point-by-point frequency analysis is impractical
- Know and apply the rules for sketching Bode plots
- Understand the significance of corner frequencies and asymptotic behavior

### 2. Apply Construction Rules to Basic Elements
- Draw Bode plots for:
  - Static gains
  - Integrators and differentiators
  - First-order systems
  - Second-order systems
  - Non-minimum phase zeros


## Import the packages:
The following cell imports the required packages. Run it before running the rest of the notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output
from IPython.display import display, clear_output
from scipy import signal

# Frequency Response Recap

## Key Concept: System Response to Sinusoidal Inputs

Linear Time-Invariant (LTI) systems have a fundamental property: when excited by a sinusoidal input, their output is also sinusoidal with:
- Same frequency as the input
- Modified amplitude
- Shifted phase

A sinusoidal input can be written as:
$$u(t) = e^{j\omega t} + e^{-j\omega t} = 2\cos(\omega t)$$

For a system with transfer function $G(s)$, the output is:
$$y(t) = G(j\omega)e^{j\omega t} + G(-j\omega)e^{-j\omega t}$$

Using Euler's formula, this simplifies to:
$$y(t) = |G(j\omega)|\cos(\omega t + \angle G(j\omega))$$

### Key Components
1. **Magnitude Response**: $|G(j\omega)|$
   - Ratio of output amplitude to input amplitude
   - How much the system amplifies/attenuates the input

2. **Phase Response**: $\angle G(j\omega)$
   - Time shift between input and output
   - How much the system delays/advances the signal

As you might expect, the magnitude and phase responses are functions of the input frequency $\omega$. The frequency response of a system is the collection of these responses for all possible input frequencies. Therefore, it is very useful to plot the magnitude and phase responses as functions of frequency, as it will give us a good understanding of how the system behaves for different input frequencies and also give us a good insight into the stability of the system.

The first intuitive thinking on how to plot the frequency response of a system is to evaluate the transfer function at different frequencies and plot the magnitude and phase of the transfer function. 

In [ ]:
# Initialize output widget for plotting
output = widgets.Output()

# Global variables to store points
frequencies = []
magnitudes = []
phases = []

def parse_polynomial(poly_str):
    """Convert string polynomial coefficients to list of floats"""
    return [float(coef.strip()) for coef in poly_str.split()]

def calculate_response(freq, num_str, den_str):
    """Calculate magnitude and phase at given frequency"""
    try:
        num = parse_polynomial(num_str)
        den = parse_polynomial(den_str)
        sys = signal.TransferFunction(num, den)
        w, mag, phase = signal.bode(sys, w=[freq])
        return mag[0], phase[0]
    except Exception as e:
        print(f"Error calculating response: {str(e)}")
        return 0, 0

def update_plots(frequency, num_str, den_str, show_full=False):
    with output:
        clear_output(wait=True)
        
        try:
            # Add new point if not already present
            if frequency not in frequencies:
                mag, phase = calculate_response(frequency, num_str, den_str)
                frequencies.append(frequency)
                magnitudes.append(mag)
                phases.append(phase)
                
                # Sort points by frequency
                sorted_indices = np.argsort(frequencies)
                frequencies[:] = [frequencies[i] for i in sorted_indices]
                magnitudes[:] = [magnitudes[i] for i in sorted_indices]
                phases[:] = [phases[i] for i in sorted_indices]
            
            # Create figure with side-by-side plots
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))
            
            # Plot measured points with lines
            if frequencies:
                ax1.semilogx(frequencies, magnitudes, 'b.-', label='Measured Response')
                ax2.semilogx(frequencies, phases, 'r.-')
                # Highlight the current frequency point
                if frequency in frequencies:
                    idx = frequencies.index(frequency)
                else:
                    idx = -1
                ax1.semilogx(frequency, magnitudes[idx], 'go', markersize=10, label='Current Frequency')
                ax2.semilogx(frequency, phases[idx], 'go', markersize=10)
            
            # Show complete Bode plot if requested
            if show_full:
                w = np.logspace(-1, 3, 10000)
                sys = signal.TransferFunction(
                    parse_polynomial(num_str),
                    parse_polynomial(den_str)
                )
                w, mag, phase = signal.bode(sys, w=w)
                
                ax1.semilogx(w, mag, 'b--', alpha=0.5, label='Complete Response', linewidth=3)
                ax2.semilogx(w, phase, 'b--', alpha=0.5, linewidth=3)
            
            # Configure magnitude plot
            ax1.grid(True)
            ax1.set_ylabel('Magnitude (dB)')
            ax1.set_xlabel('Frequency (rad/s)')
            ax1.set_title('Magnitude Response')
            if frequencies:
                ax1.legend()
            
            # Configure phase plot
            ax2.grid(True)
            ax2.set_ylabel('Phase (degrees)')
            ax2.set_xlabel('Frequency (rad/s)')
            ax2.set_title('Phase Response')
            
            # Set axis limits
            ax1.set_xlim([0.1, 1000])
            ax2.set_xlim([0.1, 1000])
            ax1.set_ylim([-60, 20])
            ax2.set_ylim([-180, 180])
            
            plt.tight_layout()
            plt.show()
            
            # Print current transfer function and frequency
            print(f"\nTransfer Function:")
            print(f"Numerator coefficients: {num_str}")
            print(f"Denominator coefficients: {den_str}")
            print(f"Current frequency: ω = {frequency:.2f} rad/s")
            mag, phase = calculate_response(frequency, num_str, den_str)
            print(f"Magnitude: {mag:.1f} dB")
            print(f"Phase: {phase:.1f}°")
                
        except Exception as e:
            print(f"Error updating plots: {str(e)}")
            print("\nPlease check your transfer function coefficients format.")
            print("Example format: '1 2 3' for 1s² + 2s + 3")

def clear_points(b):
    """Clear all points"""
    global frequencies, magnitudes, phases
    frequencies = []
    magnitudes = []
    phases = []
    update_plots(freq_slider.value, num_input.value, den_input.value)

def show_full(b):
    """Show the complete Bode plot"""
    update_plots(freq_slider.value, num_input.value, den_input.value, show_full=True)

# Create widgets
freq_slider = widgets.FloatLogSlider(
    value=1.0,
    base=10,
    min=-1,
    max=3,
    step=0.001,  # Smaller step size for smoother movement
    description='Frequency (rad/s):',
    style={'description_width': 'initial'},
    continuous_update=True
)

num_input = widgets.Text(
    value='1',
    description='Numerator:',
    style={'description_width': 'initial'},
    tooltip='Enter coefficients separated by spaces (highest power first)'
)

den_input = widgets.Text(
    value='1 1',
    description='Denominator:',
    style={'description_width': 'initial'},
    tooltip='Enter coefficients separated by spaces (highest power first)'
)

clear_button = widgets.Button(description='Clear Points')
show_full_button = widgets.Button(description='Show Full Response')

# Set up button callbacks
clear_button.on_click(clear_points)
show_full_button.on_click(show_full)

# Create layout
controls = widgets.VBox([
    widgets.HTML(value="<b>Enter transfer function coefficients:</b><br>Example: '1 2 3' for s² + 2s + 3"),
    num_input,
    den_input,
    freq_slider,
    widgets.HBox([clear_button, show_full_button])
])

# Connect the updates to widget changes
interactive_plot = interactive_output(
    update_plots, 
    {
        'frequency': freq_slider,
        'num_str': num_input,
        'den_str': den_input
    }
)

# Display everything
display(widgets.HBox([controls, output]))

# Initial plot
update_plots(freq_slider.value, num_input.value, den_input.value)